<a href="https://colab.research.google.com/github/Gaurav-phatkare/Amazon-Product-Reviews-Sentiment-Analysis/blob/main/Amazon_Product_review_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install tensorflow==2.9.1
import tensorflow as tf
print(tf.__version__) # version needed for Tensorflow_text library

2.13.0


In [2]:
# !pip install tensorflow_text

In [4]:
import tensorflow_text as text
import tensorflow_hub as hub

In [5]:
import tensorflow_datasets as tfds
ds = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', split='train', shuffle_files=True)

In [6]:
import tensorflow as tf
assert isinstance(ds, tf.data.Dataset)
print(ds)

<_PrefetchDataset element_spec={'data': {'customer_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'helpful_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'marketplace': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_category': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_parent': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_body': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_date': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_headline': TensorSpec(shape=(), dtype=tf.string, name=None), 'review_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'star_rating': TensorSpec(shape=(), dtype=tf.int32, name=None), 'total_votes': TensorSpec(shape=(), dtype=tf.int32, name=None), 'verified_purchase': TensorSpec(shape=(), dtype=tf.int64, name=None), 'vine': TensorSpec(shape=(), dtype

In [7]:
df = tfds.as_dataframe(ds)

In [8]:
df.head()

,data/customer_id,data/helpful_votes,data/marketplace,data/product_category,data/product_id,data/product_parent,data/product_title,data/review_body,data/review_date,data/review_headline,data/review_id,data/star_rating,data/total_votes,data/verified_purchase,data/vine
0,b'20980074',0,b'US',b'Mobile_Electronics',b'B00D1847NE',b'274617424',b'Teenage Mutant Ninja Turtles Boombox CD Play...,b'Does not work',b'2015-01-09',b'One Star',b'R1OVS0D6SEXPW7',1,0,0,1
1,b'779273',0,b'US',b'Mobile_Electronics',b'B00KMO6DYG',b'397452138',b'4 Gauge Amp Kit Amplifier Install Wiring Com...,b'This is a great wiring kit i used it to set ...,b'2015-08-06',b'Great kit',b'R9VSD0ET8FERB',4,0,0,1
2,b'15410531',0,b'US',b'Mobile_Electronics',b'B000GWLL0K',b'948304826',b'Travel Wall Charger fits Creative Zen Vision...,b'It works great so much faster than USB charg...,b'2007-03-15',b'A/C Charger for Creative Zen Vision M',b'R3ISXCZHWLJLBH',5,0,0,1
3,b'27389005',0,b'US',b'Mobile_Electronics',b'B008L3JE6Y',b'466340015',b'High Grade Robust 360\xc2\xb0 Adjustable Car...,b'This product was purchased to hold a monitor...,b'2013-07-30',b'camera stand',b'R1TWVUDOFJSQAW',5,0,0,1
4,b'2663569',0,b'US',b'Mobile_Electronics',b'B00GHZS4SC',b'350592810',b'HDE Multifunctional Bluetooth FM Audio Car K...,"b""it works but it has really bad sound quality...",b'2014-12-31',b'bad sound quality',b'R2PEOEUR1LP0GH',3,0,0,1


In [9]:
df.shape

(104975, 15)

In [10]:
df["Sentiment"] = df["data/star_rating"].apply(lambda score: "positive" if score >= 3 else "negative")
df['Sentiment'] = df['Sentiment'].map({'positive':1, 'negative':0})
df['short_review'] =df['data/review_body'].str.decode("utf-8")
df = df[["short_review", "Sentiment"]]

In [11]:
# Dropping last n rows using drop
n = 54975
df.drop(df.tail(n).index,
        inplace = True)

<ipython-input-11-d12344173841>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(df.tail(n).index,


In [12]:
reviews = df['short_review'].values.tolist()
labels = df['Sentiment'].tolist()

In [13]:
reviews

['Does not work',
 'This is a great wiring kit i used it to set up my Pyle 2000 watt amp to 2 alpine subs and worked just fine. im not sure about the power cord and its length because my battery is in the back but the ground cord is a bit short and thats its only flaw and that why i gave it 4 stars. great kit',
 'It works great so much faster than USB charger..Buy it! You will be glad you did!',
 'This product was purchased to hold a monitor on a desk, which is connected to a security camera at a door of our business. It serves our purpose perfectly.',
 "it works but it has really bad sound quality. the bass doesn't work almost at all",
 "Excellent product !!!<br />Easy to install and performs flawlessly.<br />Video and audio is clear and GPS map pinpoints the exact vehicle location.<br />I'm thinking of getting another one for my wifes' car.<br />Dealer product support is excellent too !!!",
 "This product works great so far. I have been using it for a month and there have been no pro

## Split the dataset into train and val

In [14]:
from sklearn.model_selection import train_test_split

train_sentances, val_sentances, train_labels, val_labels = train_test_split(df['short_review'].to_numpy(),
                                                                            df['Sentiment'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [15]:
# check the length
len(train_sentances), len(val_sentances)

(45000, 5000)

## Converting text into numbers

###Text Vectorizaion

In [16]:
import tensorflow as tf

from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = 'int',
                                    output_sequence_length = None)

In [17]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentances])/len(train_sentances))

64

In [18]:
max_vocab_length = 20000
max_length = 64

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [19]:
text_vectorizer.adapt(train_sentances)

In [20]:
# Create sample sentence and tokenize it
sample_sentence = "The product was too good than my expectations"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[  2,  36,  19, 108,  30,  62,  11, 982,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])>

In [21]:
#Choose a random sentence from the training dataset and tokenize it
import random
random_sentance = random.choice(train_sentances)
print(f"Original text:\n{random_sentance}\
      \n\nVectorized version:")
text_vectorizer([random_sentance])

Original text:
It works just fine, but it required some altering. The original product was a couple millimeters too big, but enough to make it not fit properly. Good price though.      

Vectorized version:


<tf.Tensor: shape=(1, 64), dtype=int64, numpy=
array([[    4,    56,    37,   165,    20,     4,  1206,    99, 13541,
            2,   353,    36,    19,     7,   331, 10106,   108,   261,
           20,   171,     6,   153,     4,    17,   102,   639,    30,
           68,   215,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0]])>

In [22]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary()

In [23]:
words_in_vocab[:5]

['', '[UNK]', 'the', 'i', 'it']

In [24]:
words_in_vocab[-5:]

['idorks', 'identico', 'identically', 'ideapad', 'ideacentre']

In [25]:
len(words_in_vocab)

20000

## create embedding using an embedding layer

In [26]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim = max_vocab_length,
                                      output_dim = 128,
                                      embeddings_initializer = 'uniform',
                                      input_length = max_length,
                                      name = 'embedding_1')

In [27]:
# Get a random sentence from training set
random_sentance = random.choice(train_sentances)
print(f"Original text:\n{random_sentance}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentance]))
sample_embed

Original text:
Everything I expected. My TomTom fits fine as well as the cable. Very pleased with the case and the price.      

Embedded version:


<tf.Tensor: shape=(1, 64, 128), dtype=float32, numpy=
array([[[ 0.02700094, -0.01682483,  0.00473648, ..., -0.04842658,
         -0.03995174,  0.02397213],
        [-0.0048058 ,  0.04486765, -0.02519393, ...,  0.02659268,
          0.04467485,  0.0323605 ],
        [-0.00783036,  0.0460988 ,  0.00937785, ...,  0.00794622,
          0.04967376, -0.0067549 ],
        ...,
        [-0.0122037 , -0.03019127,  0.04914934, ...,  0.04423639,
         -0.04320024, -0.03136003],
        [-0.0122037 , -0.03019127,  0.04914934, ...,  0.04423639,
         -0.04320024, -0.03136003],
        [-0.0122037 , -0.03019127,  0.04914934, ...,  0.04423639,
         -0.04320024, -0.03136003]]], dtype=float32)>

# Model 0: Baseline model

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


# Create tokenization and modelling pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf',MultinomialNB())
])

# fit the pipeline to the training data

model_0.fit(train_sentances, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
baseline_score = model_0.score(val_sentances, val_labels)

In [30]:
baseline_score

0.7978

In [31]:
baseline_preds = model_0.predict(val_sentances)
baseline_preds[:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [32]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [33]:
baseline_results = calculate_results(val_labels, baseline_preds)

In [34]:
baseline_results

{'accuracy': 79.78,
 'precision': 0.8338266657514186,
 'recall': 0.7978,
 'f1': 0.7308971860355469}

# model 1: A simple dense model

In [35]:
# Build model with the functional API
from tensorflow.keras import layers

inputs = layers.Input(shape = (1, ), dtype = 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)

model_1 = tf.keras.Model(inputs, outputs, name = 'model_1_dense')

In [36]:
# compile a model

model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 64, 128)           2560000   
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2560129 (9.77 MB)
Trainable params: 256

In [38]:
# fit the model

model_1_history = model_1.fit(train_sentances,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 63s 42ms/step - loss: 0.3592 - accuracy: 0.8418 - val_loss: 0.2700 - val_accuracy: 0.8854
Epoch 2/5
1407/1407 [==============================] - 13s 9ms/step - loss: 0.2488 - accuracy: 0.8942 - val_loss: 0.2501 - val_accuracy: 0.8932
Epoch 3/5
1407/1407 [==============================] - 11s 8ms/step - loss: 0.2067 - accuracy: 0.9140 - val_loss: 0.2465 - val_accuracy: 0.8960
Epoch 4/5
1407/1407 [==============================] - 10s 7ms/step - loss: 0.1719 - accuracy: 0.9323 - val_loss: 0.2491 - val_accuracy: 0.8974
Epoch 5/5
1407/1407 [==============================] - 10s 7ms/step - loss: 0.1426 - accuracy: 0.9471 - val_loss: 0.2580 - val_accuracy: 0.8948


In [39]:
model_1.evaluate(val_sentances, val_labels)

157/157 [==============================] - 1s 3ms/step - loss: 0.2580 - accuracy: 0.8948


[0.25799670815467834, 0.8948000073432922]

In [40]:
# model 1 prediction probability

model_1_pred_prob = model_1.predict(val_sentances)

157/157 [==============================] - 1s 3ms/step


In [41]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_prob))

In [42]:
model_1_preds

<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [43]:
# Calculate model_1 metrics
model_1_results = calculate_results(val_labels,
                                    model_1_preds)
model_1_results

{'accuracy': 89.48,
 'precision': 0.8915664803194004,
 'recall': 0.8948,
 'f1': 0.8918060016657496}

# model 2: LSTM

In [44]:
from tensorflow.keras import layers

model_2_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 256,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_2_model')

# build a model
inputs = layers.Input(shape =(1, ), dtype = 'string')
x = text_vectorizer(inputs)
x  = model_2_embedding(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name = 'model_2_LSTM')

(None, 64, 256)
(None, 64)


In [45]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [46]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_2_model (Embeddi  (None, 64, 256)           5120000   
 ng)                                                             
                                                                 
 lstm (LSTM)                 (None, 64)                82176     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5202241 (19.84 MB)
Trainable params: 520

In [47]:
# Fit model
model_2_history = model_2.fit(train_sentances,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 69s 46ms/step - loss: 0.5032 - accuracy: 0.7768 - val_loss: 0.3850 - val_accuracy: 0.8306
Epoch 2/5
1407/1407 [==============================] - 20s 14ms/step - loss: 0.3799 - accuracy: 0.8437 - val_loss: 0.3091 - val_accuracy: 0.8684
Epoch 3/5
1407/1407 [==============================] - 17s 12ms/step - loss: 0.2511 - accuracy: 0.8990 - val_loss: 0.2719 - val_accuracy: 0.8858
Epoch 4/5
1407/1407 [==============================] - 15s 11ms/step - loss: 0.1820 - accuracy: 0.9293 - val_loss: 0.2694 - val_accuracy: 0.8860
Epoch 5/5
1407/1407 [==============================] - 15s 10ms/step - loss: 0.1363 - accuracy: 0.9490 - val_loss: 0.3225 - val_accuracy: 0.8792


In [48]:
# make some predictions

model_2_pred_probs = model_2.predict(val_sentances)

157/157 [==============================] - 1s 3ms/step


In [49]:
model_2_pred = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [50]:
# Calculate LSTM model results
model_2_results = calculate_results(val_labels,
                                    model_2_pred)
model_2_results

{'accuracy': 87.92,
 'precision': 0.8838967841318305,
 'recall': 0.8792,
 'f1': 0.8810571951644228}

# Model 3: GRU

In [51]:
# import tensorflow as tf
from tensorflow.keras import layers

model_3_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_3_mode')


# build a Rnn with GRU cell

inputs  = layers.Input(shape=(1,), dtype = 'string')
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layers.GRU(64, dropout=0.25)(x)
# x = layers.Dense(64, activation = 'relu')(x)
x = layers.Dense(64, activation = 'relu')(x)
# model.add(keras.layers.Dropout(0.4))
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name = 'model_3_GRU')

In [52]:
# compile the Model

model_3.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [53]:
# summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_3_mode (Embeddin  (None, 64, 128)           2560000   
 g)                                                              
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                

In [54]:


# fit the model

model_3_history = model_3.fit(train_sentances,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 52s 34ms/step - loss: 0.4158 - accuracy: 0.8232 - val_loss: 0.2534 - val_accuracy: 0.8914
Epoch 2/5
1407/1407 [==============================] - 17s 12ms/step - loss: 0.2291 - accuracy: 0.9071 - val_loss: 0.2489 - val_accuracy: 0.8948
Epoch 3/5
1407/1407 [==============================] - 16s 11ms/step - loss: 0.1793 - accuracy: 0.9288 - val_loss: 0.2614 - val_accuracy: 0.8928
Epoch 4/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.1347 - accuracy: 0.9497 - val_loss: 0.3103 - val_accuracy: 0.8862
Epoch 5/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.0980 - accuracy: 0.9641 - val_loss: 0.3717 - val_accuracy: 0.8808


In [55]:
# Epoch 1/5
# 1407/1407 [==============================] - 48s 32ms/step - loss: 0.4128 - accuracy: 0.8209 - val_loss: 0.2496 - val_accuracy: 0.8944
# Epoch 2/5
# 1407/1407 [==============================] - 15s 11ms/step - loss: 0.2227 - accuracy: 0.9084 - val_loss: 0.2412 - val_accuracy: 0.8984
# Epoch 3/5
# 1407/1407 [==============================] - 15s 11ms/step - loss: 0.1578 - accuracy: 0.9385 - val_loss: 0.2747 - val_accuracy: 0.8898
# Epoch 4/5
# 1407/1407 [==============================] - 13s 10ms/step - loss: 0.1037 - accuracy: 0.9613 - val_loss: 0.3157 - val_accuracy: 0.8910
# Epoch 5/5
# 1407/1407 [==============================] - 14s 10ms/step - loss: 0.0687 - accuracy: 0.9758 - val_loss: 0.4189 - val_accuracy: 0.8876

In [56]:
# evaluate the model
model_3.evaluate(val_sentances)

157/157 [==============================] - 1s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

In [57]:
# Make predictions on the validation data

model_3_pred_probs = model_3.predict(val_sentances)
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred

157/157 [==============================] - 1s 4ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 0., 1.], dtype=float32)>

In [58]:
# Calculate LSTM model results
model_3_results = calculate_results(val_labels,
                                    model_3_pred)
model_3_results

{'accuracy': 88.08,
 'precision': 0.881171283328734,
 'recall': 0.8808,
 'f1': 0.8809812377281127}

# Model 4 : Bidirectional RNN Model

In [59]:
from tensorflow.keras import layers

model_4_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_4_model')

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)

x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

In [60]:
# compile the model

model_4.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [61]:
model_4.summary()

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_4_model (Embeddi  (None, 64, 128)           2560000   
 ng)                                                             
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                             

In [62]:
# Fit the model
model_4_history = model_4.fit(train_sentances,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 57s 38ms/step - loss: 0.3107 - accuracy: 0.8681 - val_loss: 0.2552 - val_accuracy: 0.8962
Epoch 2/5
1407/1407 [==============================] - 20s 14ms/step - loss: 0.2170 - accuracy: 0.9131 - val_loss: 0.2474 - val_accuracy: 0.8990
Epoch 3/5
1407/1407 [==============================] - 18s 13ms/step - loss: 0.1645 - accuracy: 0.9361 - val_loss: 0.2797 - val_accuracy: 0.8860
Epoch 4/5
1407/1407 [==============================] - 18s 13ms/step - loss: 0.1151 - accuracy: 0.9567 - val_loss: 0.3231 - val_accuracy: 0.8818
Epoch 5/5
1407/1407 [==============================] - 18s 13ms/step - loss: 0.0833 - accuracy: 0.9695 - val_loss: 0.3716 - val_accuracy: 0.8886


In [63]:
# Make predictions
model_4_pred_probs = model_4.predict(val_sentances)
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred

157/157 [==============================] - 2s 7ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [64]:
# Calculate model 4 results
model_4_results = calculate_results(val_labels,
                                    model_4_pred)
model_4_results

{'accuracy': 88.86,
 'precision': 0.8872602249055144,
 'recall': 0.8886,
 'f1': 0.8878450690503394}

# Model 5: Conv1D

In [65]:
# test out the embedding , 1D convolutional and max pooling

embedding_test = embedding(text_vectorizer(["Packaging of product was too bad"]))

conv_1d = layers.Conv1D(filters = 32, kernel_size = 5, activation = 'relu')
conv_1d_output = conv_1d(embedding_test)

max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 64, 128]), TensorShape([1, 60, 32]), TensorShape([1, 32]))

In [66]:
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]

(<tf.Tensor: shape=(1, 64, 128), dtype=float32, numpy=
 array([[[-0.04784365, -0.0650325 , -0.12291285, ..., -0.02730245,
          -0.02577849, -0.08949862],
         [-0.0580535 , -0.10558667, -0.1328013 , ..., -0.0771806 ,
          -0.07982657, -0.11582603],
         [-0.06651627, -0.23512976, -0.2292752 , ..., -0.07200687,
          -0.08490938, -0.18637015],
         ...,
         [-0.08965454, -0.2775475 , -0.25046456, ..., -0.0859485 ,
          -0.15653276, -0.20958346],
         [-0.08965454, -0.2775475 , -0.25046456, ..., -0.0859485 ,
          -0.15653276, -0.20958346],
         [-0.08965454, -0.2775475 , -0.25046456, ..., -0.0859485 ,
          -0.15653276, -0.20958346]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 60, 32), dtype=float32, numpy=
 array([[[0.        , 0.00370973, 0.        , ..., 0.02502503,
          0.0178159 , 0.15855919],
         [0.        , 0.0014108 , 0.        , ..., 0.        ,
          0.01571468, 0.17887524],
         [0.        , 0.08071481, 0.08

In [67]:
# build a model
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer = 'uniform',
                                     input_length = max_length,
                                     name = 'embedding_5')


from tensorflow.keras import layers

inputs = layers.Input(shape =(1, ), dtype=tf.string)
x = text_vectorizer(inputs)
x = model_5_embedding(x)

x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
# x = layers.Dense(64, activation = 'relu', name = 'extraLayer')(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

In [68]:
# compile the model

model_5.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 64)                0         
 Vectorization)                                                  
                                                                 
 embedding_5 (Embedding)     (None, 64, 128)           2560000   
                                                                 
 conv1d_1 (Conv1D)           (None, 60, 32)            20512     
                                                                 
 global_max_pooling1d_2 (Gl  (None, 32)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_5 (Dense)             (None, 1)              

In [69]:
# Fit the model
model_5_history = model_5.fit(train_sentances,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 49s 33ms/step - loss: 0.3164 - accuracy: 0.8616 - val_loss: 0.2550 - val_accuracy: 0.8924
Epoch 2/5
1407/1407 [==============================] - 12s 9ms/step - loss: 0.2070 - accuracy: 0.9155 - val_loss: 0.2564 - val_accuracy: 0.8926
Epoch 3/5
1407/1407 [==============================] - 11s 8ms/step - loss: 0.1327 - accuracy: 0.9505 - val_loss: 0.2875 - val_accuracy: 0.8884
Epoch 4/5
1407/1407 [==============================] - 11s 8ms/step - loss: 0.0738 - accuracy: 0.9766 - val_loss: 0.3314 - val_accuracy: 0.8906
Epoch 5/5
1407/1407 [==============================] - 11s 8ms/step - loss: 0.0381 - accuracy: 0.9899 - val_loss: 0.3797 - val_accuracy: 0.8852


In [70]:
# # Fit the model
# model_5_history = model_5.fit(train_sentances,
#                               train_labels,
#                               epochs = 5,
#                               validation_data = (val_sentances, val_labels))

In [71]:
# Make Predictions
model_5_pred_prob = model_5.predict(val_sentances)
model_5_pred = tf.squeeze(tf.round(model_5_pred_prob))
model_5_pred

157/157 [==============================] - 1s 3ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([0., 1., 1., ..., 1., 0., 1.], dtype=float32)>

In [72]:
# Calculate 5th model results
model_5_results = calculate_results(val_labels,
                                    model_5_pred)
model_5_results

{'accuracy': 88.52,
 'precision': 0.8836679809824549,
 'recall': 0.8852,
 'f1': 0.8843286121247045}

# Model 6: pretrain models from Tensorflow Hub

In [73]:
import tensorflow_hub as hub

embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')


In [74]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [75]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [76]:
# Train a classifier on top of pretrained embeddings
model_6_history = model_6.fit(train_sentances,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentances, val_labels))

Epoch 1/5
1407/1407 [==============================] - 47s 29ms/step - loss: 0.2865 - accuracy: 0.8746 - val_loss: 0.2573 - val_accuracy: 0.8924
Epoch 2/5
1407/1407 [==============================] - 43s 30ms/step - loss: 0.2558 - accuracy: 0.8898 - val_loss: 0.2535 - val_accuracy: 0.8938
Epoch 3/5
1407/1407 [==============================] - 40s 28ms/step - loss: 0.2497 - accuracy: 0.8922 - val_loss: 0.2481 - val_accuracy: 0.8960
Epoch 4/5
1407/1407 [==============================] - 41s 29ms/step - loss: 0.2434 - accuracy: 0.8951 - val_loss: 0.2468 - val_accuracy: 0.9010
Epoch 5/5
1407/1407 [==============================] - 40s 29ms/step - loss: 0.2371 - accuracy: 0.8982 - val_loss: 0.2419 - val_accuracy: 0.8962


In [77]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_sentances)
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds

157/157 [==============================] - 4s 20ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [78]:
# Calculate model 6 performance metrics
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

{'accuracy': 89.62,
 'precision': 0.8937475750287,
 'recall': 0.8962,
 'f1': 0.8944811486131653}

# Model 7: 10 percent data on model6

In [79]:
import numpy as np
train_sentences_90_percent, train_sentences_10_percent, train_labels_90_percent, train_labels_10_percent = train_test_split(np.array(train_sentances),
                                                                                                                            train_labels,
                                                                                                                            test_size=0.1,
                                                                                                                            random_state=42)

In [80]:
len(train_sentences_10_percent),len(train_labels_10_percent)

(4500, 4500)

In [81]:
len(train_sentences_90_percent)

40500

In [82]:
import pandas as pd
pd.Series(train_labels_10_percent).value_counts()

1    3419
0    1081
dtype: int64

In [83]:

model_7 = tf.keras.models.clone_model(model_6)

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [84]:
# Fit the model to 10% of the training data
model_7_history = model_7.fit(x=train_sentences_10_percent,
                              y=train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentances, val_labels),)

Epoch 1/5
141/141 [==============================] - 11s 45ms/step - loss: 0.4416 - accuracy: 0.8007 - val_loss: 0.3234 - val_accuracy: 0.8554
Epoch 2/5
141/141 [==============================] - 8s 57ms/step - loss: 0.2887 - accuracy: 0.8818 - val_loss: 0.2759 - val_accuracy: 0.8814
Epoch 3/5
141/141 [==============================] - 6s 42ms/step - loss: 0.2607 - accuracy: 0.8942 - val_loss: 0.2722 - val_accuracy: 0.8786
Epoch 4/5
141/141 [==============================] - 7s 46ms/step - loss: 0.2512 - accuracy: 0.8964 - val_loss: 0.2690 - val_accuracy: 0.8816
Epoch 5/5
141/141 [==============================] - 7s 53ms/step - loss: 0.2446 - accuracy: 0.8998 - val_loss: 0.2679 - val_accuracy: 0.8832


In [85]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(val_sentances)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds

157/157 [==============================] - 6s 33ms/step


<tf.Tensor: shape=(5000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [86]:
model_7_results = calculate_results(val_labels, model_7_preds)
model_7_results

{'accuracy': 88.32,
 'precision': 0.8794484638099735,
 'recall': 0.8832,
 'f1': 0.8802728976229992}

# Model 8:BERT Model

In [87]:
# !pip install tensorflow_text

In [96]:
# import tensorflow_hub as hub
# # import tensorflow_text as text

# https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [102]:
def sentence_embedding(sentences):
  preprocessed_text = bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)['pooled_output']

In [103]:
# Bert layer
inputs = tf.keras.layers.Input(shape =(), dtype = tf.string,name = 'text')
preprocess_layer = bert_preprocess(inputs)

outputs = bert_encoder(preprocess_layer)

x = layers.Dropout(0.1, name = "dropout_1")(outputs['pooled_output'])
x = layers.Dense(1, activation='sigmoid', name = 'output')(x)


BERT_model = tf.keras.Model(inputs=[inputs],outputs = [x])

In [104]:
BERT_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [105]:
# compile a model
BERT_model.compile(loss = 'binary_crossentropy',
                   optimizer = tf.keras.optimizers.Adam(),
                   metrics=['accuracy'])

In [ ]:
#fit the model
Bert_model_history = BERT_model.fit(train_sentances,
                              train_labels,
                              epochs=3,
                              validation_data=(val_sentances, val_labels))

Epoch 1/3
1407/1407 [==============================] - 596s 422ms/step - loss: 0.4687 - accuracy: 0.7802 - val_loss: 0.3931 - val_accuracy: 0.8104
Epoch 2/3
1407/1407 [==============================] - 591s 420ms/step - loss: 0.3924 - accuracy: 0.8187 - val_loss: 0.3514 - val_accuracy: 0.8400
Epoch 3/3
  76/1407 [>.............................] - ETA: 8:00 - loss: 0.3750 - accuracy: 0.8285

# Compare the all results

In [89]:
# Combine model results into a DataFrame
all_results_models = pd.DataFrame({"0_baseline": baseline_results,
                                  "1_simple_dense": model_1_results,
                                  "2_lstm": model_2_results,
                                  "3_gru": model_3_results,
                                  "4_bidirectional": model_4_results,
                                  "5_conv1d": model_5_results,
                                  "6_tf_hub_sentence_encoder": model_6_results,
                                  "7_tf_hub_10_percent_data": model_7_results})
all_results_models = all_results_models.transpose()
all_results_models

,accuracy,precision,recall,f1
0_baseline,79.78,0.833827,0.7978,0.730897
1_simple_dense,89.48,0.891566,0.8948,0.891806
2_lstm,87.92,0.883897,0.8792,0.881057
3_gru,88.08,0.881171,0.8808,0.880981
4_bidirectional,88.86,0.887260,0.8886,0.887845
5_conv1d,88.52,0.883668,0.8852,0.884329
6_tf_hub_sentence_encoder,89.62,0.893748,0.8962,0.894481
7_tf_hub_10_percent_data,88.32,0.879448,0.8832,0.880273


In [90]:
def predict_on_sentence(model, sentence):
  """
  Uses model to make a prediction on sentence.

  Returns the sentence, the predicted label and the prediction probability.
  """
  pred_prob = model.predict([sentence])
  pred_label = tf.squeeze(tf.round(pred_prob)).numpy()
  print(f"Pred: {pred_label}", "Positive" if pred_label > 0 else "Negative", f"Prob: {pred_prob[0][0]}")
  print(f"Text:\n{sentence}")

In [91]:
predict_on_sentence(model_6, "I will not recommend this to anyone ")

1/1 [==============================] - 0s 42ms/step
Pred: 0.0 Negative Prob: 0.19708631932735443
Text:
I will not recommend this to anyone 
